Untuk melihat bagaimana efektifnya transfer learning, kita akan mencoba menggunakan transfer learning pada dataset cheesman yang kita kerjakan pada project sebelumnya.

In [1]:
!wget --no-check-certificate \
  https://github.com/dicodingacademy/assets/raw/main/ml_pengembangan_academy/Chessman-image-dataset.zip \
  -O /tmp/Chessman-image-dataset.zip

--2022-11-16 01:49:36--  https://github.com/dicodingacademy/assets/raw/main/ml_pengembangan_academy/Chessman-image-dataset.zip
Resolving github.com (github.com)... 140.82.121.4
Connecting to github.com (github.com)|140.82.121.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/dicodingacademy/assets/main/ml_pengembangan_academy/Chessman-image-dataset.zip [following]
--2022-11-16 01:49:36--  https://raw.githubusercontent.com/dicodingacademy/assets/main/ml_pengembangan_academy/Chessman-image-dataset.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 60684125 (58M) [application/zip]
Saving to: ‘/tmp/Chessman-image-dataset.zip’

/tmp/Chessman-image 100%[===================>]  57.87M  --.-KB/s    in 

Kemudian, kita buat direktori untuk dataset kita untuk digunakan pada ImageDataGenerator. 

In [2]:
#unzip terlebih dahulu
import os
import zipfile
from tensorflow.keras.preprocessing.image import ImageDataGenerator
local_zip = '/tmp/Chessman-image-dataset.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp')
zip_ref.close()

#tentukan direktori
train_dir = os.path.join('/tmp/Chessman-image-dataset/Chess')

#imagedatagenerator
train_datagen = ImageDataGenerator(rescale=1./255,
    rotation_range=20,
    zoom_range=0.2,
    shear_range=0.2,
    fill_mode = 'nearest',
    validation_split=0.1) # set validation split

Bagi dataset menjadi data training dan data validasi.

In [3]:
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=8,
    class_mode='categorical',
    subset='training') # set as training data
validation_generator = train_datagen.flow_from_directory(
    train_dir, # same directory as training data
    target_size=(150, 150),
    batch_size=16,
    class_mode='categorical',
    subset='validation')

Found 499 images belonging to 6 classes.
Found 52 images belonging to 6 classes.


Nah, disini kita mulai mengimplementasikan transfer learning. Untuk model yang kita pilih sebagai model dasar transfer learning adalah ResNet152V2. Model ResNet152V2 memiliki sebanyak 152 layer dan tersedia di library keras.

Untuk mengimplementasikan transfer learning sangatlah mudah seperti kode di bawah. Kita hanya perlu menambahkan 2 buah baris kode berbeda. Layer pertama pada model kita adalah model yang kita pakai untuk transfer learning. Kita cukup memanggil kelas ResNet152V2 dan mengisi parameter sebagai berikut:

1. Weight : ini adalah bobot atau parameter seperti yang telah dibahas pada kelas machine learning pemula. Untuk parameter weight kita mengisi nilai ‘imagenet’. Artinya kita ingin menggunakan model ResNet152V2 yang telah dilatih pada dataset imagenet. Imagenet adalah sebuah database raksasa yang berisi lebih dari 14 juta gambar.

2. Include_top : parameter ini bernilai boolean. Maksud dari parameter ini apabila kita ingin tetap memakai layer terakhir/layer prediksi dari model resnet. Kita isi false karena kita memakai model resnet untuk memprediksi dataset chessman bukan imagenet.

3. Input_tensor : sesuai namanya parameter ini menspesifikasikan ukuran dari input.

In [4]:
import tensorflow as tf
from tensorflow.keras.layers import Input
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications import ResNet152V2
model = tf.keras.models.Sequential([
    ResNet152V2(weights="imagenet", include_top=False, input_tensor=Input(shape=(150, 150, 3))),
    # tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Flatten(), 
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(6, activation='softmax')  
])
model.layers[0].trainable = False

234545216/234545216 [==============================] - 8s 0us/step


Lanjutkan dengan menentukan optimizer, loss, serta metrik yang ingin digunakan pada model.

In [5]:
model.compile(optimizer=tf.optimizers.Adam(),
              loss='categorical_crossentropy',
              metrics = ['accuracy'])

Terakhir kita dapat melakukan pelatihan model. 

In [6]:
history = model.fit(train_generator,
                              validation_data=validation_generator,
                              epochs=50,
                              verbose=2)

Epoch 1/50
63/63 - 29s - loss: 8.8984 - accuracy: 0.4529 - val_loss: 3.5441 - val_accuracy: 0.5769 - 29s/epoch - 467ms/step
Epoch 2/50
63/63 - 10s - loss: 2.0014 - accuracy: 0.6754 - val_loss: 2.0098 - val_accuracy: 0.5962 - 10s/epoch - 166ms/step
Epoch 3/50
63/63 - 10s - loss: 1.5972 - accuracy: 0.7275 - val_loss: 1.3753 - val_accuracy: 0.7692 - 10s/epoch - 161ms/step
Epoch 4/50
63/63 - 10s - loss: 1.0407 - accuracy: 0.7715 - val_loss: 1.4824 - val_accuracy: 0.5962 - 10s/epoch - 159ms/step
Epoch 5/50
63/63 - 10s - loss: 0.9130 - accuracy: 0.7996 - val_loss: 1.9927 - val_accuracy: 0.6346 - 10s/epoch - 159ms/step
Epoch 6/50
63/63 - 10s - loss: 0.9909 - accuracy: 0.7916 - val_loss: 1.4061 - val_accuracy: 0.6346 - 10s/epoch - 161ms/step
Epoch 7/50
63/63 - 11s - loss: 0.8133 - accuracy: 0.8136 - val_loss: 1.4058 - val_accuracy: 0.7115 - 11s/epoch - 176ms/step
Epoch 8/50
63/63 - 10s - loss: 0.7522 - accuracy: 0.8317 - val_loss: 1.2607 - val_accuracy: 0.6923 - 10s/epoch - 161ms/step
Epoch 9/

Tingkat akurasi menunjukkan hasil yang jauh lebih baik dibandingkan ketika melatih model sendiri dari awal. Akurasi dari model meningkat 40% menjadi 70 pada epoch terakhir. Hasil ini sangat luar biasa mengingat setiap kelas pada sampel kita sangat sedikit yaitu kurang dari 100 buah sampel.